In [23]:
from googleapiclient.discovery import build
import pandas as pd  
from datetime import datetime
import pytz
import json
import os

In [24]:
file_path = "api_key.txt"

if not os.path.exists(file_path):
    print(f"Error: '{file_path}' not found in the directory.")
else:
    # Open the file and read the API key
    with open(file_path, 'r', encoding='utf-8') as file:
        api_key = file.read()

In [25]:
channel_id = "UCBa659QWEk1AI4Tg--mrJ2A"

In [26]:
api_service_name = "youtube"
api_version = "v3"

youtube = build(
    api_service_name, api_version, developerKey=api_key
)


In [27]:
def get_uploads_playlist_id(channel_id):
    request = youtube.channels().list(
        part="snippet,contentDetails,statistics",
        id=channel_id
    )

    response = request.execute()

    uploads_id = response["items"][0]["contentDetails"]["relatedPlaylists"]["uploads"]

    return uploads_id


uploads_playlist_id = get_uploads_playlist_id(channel_id)
print(uploads_playlist_id)

UUBa659QWEk1AI4Tg--mrJ2A


In [28]:
def get_video_ids(youtube, playlist_id):
    
    video_ids = []
    
    request = youtube.playlistItems().list(
        part="snippet,contentDetails",
        playlistId=playlist_id,
        maxResults = 50
    )
    response = request.execute()
    
    for item in response['items']:
        video_ids.append(item['contentDetails']['videoId'])
        
    next_page_token = response.get('nextPageToken')
    while next_page_token is not None:
        request = youtube.playlistItems().list(
                    part='contentDetails',
                    playlistId = playlist_id,
                    maxResults = 50,
                    pageToken = next_page_token)
        response = request.execute()

        for item in response['items']:
            video_ids.append(item['contentDetails']['videoId'])

        next_page_token = response.get('nextPageToken')
        
    return video_ids


video_ids = get_video_ids(youtube, uploads_playlist_id)

print(len(video_ids))

731


In [29]:
def get_video_details(youtube, video_ids):
    all_video_info = []

    for i in range(0, len(video_ids), 50):
        request = youtube.videos().list(
            part="snippet,contentDetails,statistics",
            id=','.join(video_ids[i:i+50])
        )

        response = request.execute()

        for video in response["items"]:
            stats_to_keep = {
                "snippet": ["title", "thumbnails", "publishedAt"]
            }

            video_info = {}
            video_info["video_id"] = video["id"]

            for k in stats_to_keep.keys():
                for v in stats_to_keep[k]:
                    try :
                        video_info[v] = video[k][v]
                    except:
                        video_info[v] = None

            all_video_info.append(video_info)

    return all_video_info

video_info = get_video_details(youtube, video_ids)
pd.DataFrame(video_info)

,video_id,title,thumbnails,publishedAt
0,7DKv5H5Frt0,"After ten years, it's time to stop weekly videos.",{'default': {'url': 'https://i.ytimg.com/vi/7D...,2024-01-01T16:00:06Z
1,F0JDK_71yDg,People are going to be angry about pylons.,{'default': {'url': 'https://i.ytimg.com/vi/F0...,2023-12-25T16:00:08Z
2,Zl_5LT2fzak,Why the government drops flies on California,{'default': {'url': 'https://i.ytimg.com/vi/Zl...,2023-12-18T16:00:18Z
3,hNZy603as5w,A robot just swapped my electric car's battery,{'default': {'url': 'https://i.ytimg.com/vi/hN...,2023-12-11T16:00:31Z
4,pU9sHwNKc2c,Why don't subtitles match dubbing?,{'default': {'url': 'https://i.ytimg.com/vi/pU...,2023-12-04T16:00:40Z
...,...,...,...,...
726,dK-pMy_jOKI,On A Pirate Ship - Jay Foreman feat. Mad Cap'n...,{'default': {'url': 'https://i.ytimg.com/vi/dK...,2007-09-03T10:45:33Z
727,dNOWXi8KoOs,Fun With A Giant Slingshot,{'default': {'url': 'https://i.ytimg.com/vi/dN...,2007-04-08T13:54:51Z
728,sZ363XfT8fk,"Pastaaargh: pasta, cooked with a kettle",{'default': {'url': 'https://i.ytimg.com/vi/sZ...,2006-07-05T12:12:09Z
729,nkTyWTC9UZc,Extra Light Pancakes,{'default': {'url': 'https://i.ytimg.com/vi/nk...,2006-07-04T12:41:29Z


In [30]:
def preprocess_video_info(video_info):
    processed_video_info = []
    for video in video_info:
        upload_date = video["publishedAt"]
        dt = datetime.strptime(upload_date, "%Y-%m-%dT%H:%M:%SZ")
        year = dt.year

        if year < 2014:
            continue
        
        resolutions = ["maxres", "standard", "high", "medium", "default"]
        thumbnail = None
        resolution_i = 0
        while thumbnail == None:
            if resolutions[resolution_i] in video["thumbnails"]:
                thumbnail = video["thumbnails"][resolutions[resolution_i]]
                pass
            else:
                resolution_i += 1

        video["thumbnail"] = thumbnail
        del video["thumbnails"]

        processed_video_info.append(video)

    return processed_video_info

processed_video_info = preprocess_video_info(video_info)
pd.DataFrame(processed_video_info)


,video_id,title,publishedAt,thumbnail
0,7DKv5H5Frt0,"After ten years, it's time to stop weekly videos.",2024-01-01T16:00:06Z,{'url': 'https://i.ytimg.com/vi/7DKv5H5Frt0/ma...
1,F0JDK_71yDg,People are going to be angry about pylons.,2023-12-25T16:00:08Z,{'url': 'https://i.ytimg.com/vi/F0JDK_71yDg/ma...
2,Zl_5LT2fzak,Why the government drops flies on California,2023-12-18T16:00:18Z,{'url': 'https://i.ytimg.com/vi/Zl_5LT2fzak/ma...
3,hNZy603as5w,A robot just swapped my electric car's battery,2023-12-11T16:00:31Z,{'url': 'https://i.ytimg.com/vi/hNZy603as5w/ma...
4,pU9sHwNKc2c,Why don't subtitles match dubbing?,2023-12-04T16:00:40Z,{'url': 'https://i.ytimg.com/vi/pU9sHwNKc2c/ma...
...,...,...,...,...
662,oqbsxjmB49Q,The 134-Hour Television Show from the Arctic C...,2014-01-05T14:00:05Z,{'url': 'https://i.ytimg.com/vi/oqbsxjmB49Q/ma...
663,tMR25RSOzFo,The Secret Pattern That Stops You Copying Bank...,2014-01-04T14:23:03Z,{'url': 'https://i.ytimg.com/vi/tMR25RSOzFo/ma...
664,nYxFrMNWbtI,How To Make An Orange Peel Flamethrower,2014-01-03T14:10:58Z,{'url': 'https://i.ytimg.com/vi/nYxFrMNWbtI/ma...
665,SQ3g5gPKlDM,Why Aren't There More Helicopter Crashes In Lo...,2014-01-02T13:26:58Z,{'url': 'https://i.ytimg.com/vi/SQ3g5gPKlDM/ma...


In [31]:
print(json.dumps(processed_video_info, indent=4))

[
    {
        "video_id": "7DKv5H5Frt0",
        "title": "After ten years, it's time to stop weekly videos.",
        "publishedAt": "2024-01-01T16:00:06Z",
        "thumbnail": {
            "url": "https://i.ytimg.com/vi/7DKv5H5Frt0/maxresdefault.jpg",
            "width": 1280,
            "height": 720
        }
    },
    {
        "video_id": "F0JDK_71yDg",
        "title": "People are going to be angry about pylons.",
        "publishedAt": "2023-12-25T16:00:08Z",
        "thumbnail": {
            "url": "https://i.ytimg.com/vi/F0JDK_71yDg/maxresdefault.jpg",
            "width": 1280,
            "height": 720
        }
    },
    {
        "video_id": "Zl_5LT2fzak",
        "title": "Why the government drops flies on California",
        "publishedAt": "2023-12-18T16:00:18Z",
        "thumbnail": {
            "url": "https://i.ytimg.com/vi/Zl_5LT2fzak/maxresdefault.jpg",
            "width": 1280,
            "height": 720
        }
    },
    {
        "video_id": "hNZy6